In [10]:
from tensorflow.keras.utils import FeatureSpace
from tensorflow.keras.models import load_model
import tensorflow as tf
import pickle
import pandas as pd
import numpy as np

import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

import helper

In [3]:
reconstructed_model = load_model(parent_dir + '\\' + "model/inference_model.keras")

In [4]:
#load tfidf_vectorizer, transform the description column
fileName = parent_dir + '\\' + 'model/tfidf_vectorizer.pkl'
with open(fileName,'rb') as f:
    tfidf_object = pickle.load(f)

In [5]:
fileName = parent_dir + '\\' + 'model/price_by_hull_material.pkl'
with open(fileName,'rb') as f:
    price_by_hull_material = pickle.load(f)

average_value_hull_material = sum(price_by_hull_material.values()) / len(price_by_hull_material)

fileName = parent_dir + '\\' + 'model/price_by_fuel_type.pkl'
with open(fileName,'rb') as f:
    price_by_fuel_type = pickle.load(f)

average_value_fuel_type = sum(price_by_fuel_type.values()) / len(price_by_fuel_type)

fileName = parent_dir + '\\' + 'model/price_by_category.pkl'
with open(fileName,'rb') as f:
    price_by_category = pickle.load(f)

average_value_category = sum(price_by_category.values()) / len(price_by_category)

In [6]:
input_dict = {
    'year': 2010.0,
    'model': 'Sailing Yacht',
    'category':'Slup Boote',
    'length': 70.,
    'fuel_type': 'Diesel',
    'hull_material': 'Steel',
    'country': 'Turkey',
    'description': 'The Futuna 70 ICHTUS has been designed for a precise purpose; a shorthanded trip around the world. Designed by famous naval architect Marc Lombard, she proved to be a quick and (very) beautiful sailing yacht. She is also comfortable, fast and easy to handle despite her length, with fresh water and energy self-sufficiency.Under her new ownership since the end of 2016, ICHTUS undertook a major refit with a complete refurbishment & upgrade in 2017',
}

In [7]:
sample = helper.build_pred_dict(input_dict, tfidf_object, price_by_hull_material, price_by_fuel_type, price_by_category)

In [8]:
reconstructed_model.predict(sample)

1/1 [==============================] - 0s 405ms/step


array([[2035009.9]], dtype=float32)

In [11]:
import json
# define a class to avoid that
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)
    
payload = json.dumps(input_dict,cls=NpEncoder)

In [12]:
payload

'{"year": 2010.0, "model": "Sailing Yacht", "category": "Slup Boote", "length": 70.0, "fuel_type": "Diesel", "hull_material": "Steel", "country": "Turkey", "description": "The Futuna 70 ICHTUS has been designed for a precise purpose; a shorthanded trip around the world. Designed by famous naval architect Marc Lombard, she proved to be a quick and (very) beautiful sailing yacht. She is also comfortable, fast and easy to handle despite her length, with fresh water and energy self-sufficiency.Under her new ownership since the end of 2016, ICHTUS undertook a major refit with a complete refurbishment & upgrade in 2017"}'